In [47]:
import pandas as pd
import numpy as np

from ortools.sat.python import cp_model

In [72]:
data_file = "data/gc_250_9"
data = pd.read_csv(data_file, sep=" ", names=["vertices", "edges"])

In [73]:
data.head()

,vertices,edges
0,250,28046
1,0,1
2,0,2
3,0,3
4,0,4


In [74]:
n = data.vertices[0]
e = data.edges[0]
edge_list = np.array(data[1:])

In [75]:
class SolutionPrinter(cp_model.CpSolverSolutionCallback):
  """Print intermediate solutions."""

  def __init__(self, variables):
    self.__variables = variables
    self.__solution_count = 0
    self.__solutions = []

  def NewSolution(self):
    self.__solution_count += 1
    self.__solutions.append([self.Value(v) for v in self.__variables])

  def SolutionCount(self):
    return self.__solution_count

  def GetSolutions(self):
    return self.__solutions

In [76]:
degrees = [0] * n
G = np.zeros((n, n))
for edge in edge_list:
    e1 = edge[0]
    e2 = edge[1]
    G[e1, e2] = 1
    G[e2, e1] = 1
    degrees[e1] += 1
    degrees[e2] += 1
max_degree_node = np.argmax(degrees)

In [93]:
def greedy(degrees, G, edge_list, n):
    max_degree_order = list(reversed(np.argsort(degrees)))
    C = [-1] * n
    C[max_degree_order[0]] = 0
    max_degree_order = max_degree_order[1:]

    for cur_index in max_degree_order:

        max_col = max(C)

        used_colours = {C[i] for i in range(0, n) if G[cur_index,i] == 1 and C[i] != -1}
        all_colours = {i for i in range(0, max_col+1)}
        available_colours = all_colours.difference(used_colours)
        col_counts = [[x,C.count(x)] for x in set(C) if x != -1]

        if len(available_colours) == 0:
            C[cur_index] = max_col + 1
        elif len(available_colours) == 1:
            C[cur_index] = list(available_colours)[0]
        else:
            # Pick least used colour
            avail_col_count = list(filter(lambda x : x[0] in available_colours, col_counts))

            min_count = avail_col_count[0][1]
            min_color = avail_col_count[0][0]

            for i in range(1, len(avail_col_count)):
                if min_count < avail_col_count[i][1]:
                    min_count = avail_col_count[i][1]
                    min_color = avail_col_count[i][0]

            C[cur_index] = min_color
    
    return C

In [ ]:
def constraint_programming(degrees, G, edge_list, n):
    max_colours = 30
    min_colours = 5
    MAX_TIME = 60.0

    for num_colours in range(min_colours, max_colours + 1):

        model = cp_model.CpModel()
        solver = cp_model.CpSolver()

        # Sets a time limit of 10 seconds.
        solver.parameters.max_time_in_seconds = MAX_TIME

        # Create the variables
        C = []
        for i in range(0, n):
            C.append(model.NewIntVar(0, num_colours - 1, "c_" + str(i)))

        # Create the constraints
        for edge in edge_list:
            e1 = edge[0]
            e2 = edge[1]
            model.Add(C[e1] != C[e2])

        # symmetry breaking
        # Chose vertex with highest degreee and assign it the first colour
        model.Add(C[max_degree_node] == 0)

        for i in range(1, num_colours):
            model.Add(C[i] <= i+1);

        model.Minimize(max(C))

        # Call the solver.
        solution_printer = SolutionPrinter(C)
        status = solver.SolveWithSolutionObserver(model, solution_printer)
        #print("Number of colours: %i" % num_colours)
        print('Number of solutions found: %i' % solution_printer.SolutionCount())

        if solution_printer.SolutionCount() > 0:
            break

    # Now ensure you get solution with minimum
    solutions = solution_printer.GetSolutions()
    max_color = [max(C) for C in solutions] + 1
    best_index = np.argmin(max_color)
    node_count = max_color[best_index]
    solution = solutions[best_index]

    return solution

Number of solutions found: 0
Number of solutions found: 0


In [9]:
degrees

array([[1.],
       [4.],
       [4.],
       [3.],
       [3.],
       [2.],
       [3.],
       [2.],
       [1.],
       [1.],
       [1.],
       [3.],
       [1.],
       [3.],
       [1.],
       [2.],
       [4.],
       [5.],
       [1.],
       [1.]])

In [86]:
C

[99,
 60,
 16,
 59,
 66,
 30,
 27,
 97,
 49,
 7,
 56,
 2,
 12,
 101,
 75,
 91,
 76,
 22,
 84,
 33,
 9,
 19,
 7,
 18,
 8,
 54,
 0,
 65,
 22,
 51,
 72,
 82,
 29,
 15,
 44,
 31,
 50,
 70,
 50,
 31,
 99,
 28,
 83,
 80,
 24,
 73,
 25,
 62,
 97,
 27,
 41,
 47,
 47,
 48,
 94,
 22,
 39,
 32,
 63,
 4,
 94,
 89,
 59,
 81,
 26,
 46,
 67,
 43,
 18,
 3,
 13,
 21,
 48,
 45,
 11,
 49,
 78,
 52,
 33,
 66,
 3,
 73,
 72,
 37,
 5,
 38,
 58,
 39,
 64,
 24,
 93,
 92,
 10,
 91,
 46,
 74,
 21,
 46,
 14,
 29,
 63,
 54,
 4,
 25,
 5,
 37,
 57,
 85,
 38,
 100,
 10,
 43,
 75,
 28,
 76,
 19,
 53,
 96,
 96,
 102,
 8,
 73,
 45,
 18,
 84,
 1,
 9,
 30,
 26,
 86,
 41,
 4,
 50,
 70,
 76,
 15,
 20,
 103,
 74,
 80,
 23,
 35,
 79,
 68,
 19,
 69,
 54,
 25,
 71,
 71,
 11,
 41,
 42,
 17,
 16,
 12,
 85,
 56,
 67,
 81,
 7,
 79,
 9,
 58,
 74,
 68,
 14,
 69,
 1,
 24,
 77,
 68,
 17,
 79,
 44,
 83,
 16,
 78,
 95,
 30,
 20,
 60,
 21,
 11,
 55,
 23,
 55,
 84,
 59,
 98,
 17,
 88,
 64,
 2,
 52,
 40,
 42,
 82,
 40,
 38,
 26,
 13,
 65,
 

In [95]:
max(C) + 1

105

In [94]:
C = greedy(degrees, G, edge_list, n)

In [38]:
set

[16, 1, 2, 6, 13, 11, 3, 4, 15, 5, 7, 19, 9, 8, 18, 10, 12, 14, 0]